In [196]:
from __future__ import print_function
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

In [197]:
class MLP_G(nn.Module):
    def __init__(self, isize, nz, nc, ngf, ngpu):
        super(MLP_G, self).__init__()
        self.ngpu = ngpu

        main = nn.Sequential(
            # Z goes into a linear of size: ngf
            nn.Linear(nz, ngf),
            nn.ReLU(True),
            nn.Linear(ngf, ngf),
            nn.ReLU(True),
            nn.Linear(ngf, ngf),
            nn.ReLU(True),
            nn.Linear(ngf, nc * isize * isize),
        )
        self.main = main
        self.nc = nc
        self.isize = isize
        self.nz = nz

    def forward(self, input):
        input = input.view(input.size(0), input.size(1))
        if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else:
            output = self.main(input)
        return output.view(output.size(0), self.nc, self.isize, self.isize)

In [198]:
class MLP_D(nn.Module):
    def __init__(self, isize, nz, nc, ndf, ngpu):
        super(MLP_D, self).__init__()
        self.ngpu = ngpu

        main = nn.Sequential(
            # Z goes into a linear of size: ndf
            nn.Linear(nc * isize * isize, ndf),
            nn.ReLU(True),
            nn.Linear(ndf, ndf),
            nn.ReLU(True),
            nn.Linear(ndf, ndf),
            nn.ReLU(True),
            nn.Linear(ndf, 1),
        )
        self.main = main
        self.nc = nc
        self.isize = isize
        self.nz = nz

    def forward(self, input):
        input = input.view(input.size(0),
                           input.size(1) * input.size(2) * input.size(3))
        if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else:
            output = self.main(input)
        output = output.mean(0)
        return output.view(1)

In [199]:
class DCGAN_D(nn.Module):
    def __init__(self, isize, nz, nc, ndf, ngpu, n_extra_layers=0):
        super(DCGAN_D, self).__init__()
        self.ngpu = ngpu
        assert isize % 16 == 0, "isize has to be a multiple of 16"

        main = nn.Sequential()
        # input is nc x isize x isize
        main.add_module('initial.conv.{0}-{1}'.format(nc, ndf),
                        nn.Conv2d(nc, ndf, 4, 2, 1, bias=False))
        main.add_module('initial.relu.{0}'.format(ndf),
                        nn.LeakyReLU(0.2, inplace=True))
        csize, cndf = isize / 2, ndf

        # Extra layers
        for t in range(n_extra_layers):
            main.add_module('extra-layers-{0}.{1}.conv'.format(t, cndf),
                            nn.Conv2d(cndf, cndf, 3, 1, 1, bias=False))
            main.add_module('extra-layers-{0}.{1}.batchnorm'.format(t, cndf),
                            nn.BatchNorm2d(cndf))
            main.add_module('extra-layers-{0}.{1}.relu'.format(t, cndf),
                            nn.LeakyReLU(0.2, inplace=True))

        while csize > 4:
            in_feat = cndf
            out_feat = cndf * 2
            main.add_module('pyramid.{0}-{1}.conv'.format(in_feat, out_feat),
                            nn.Conv2d(in_feat, out_feat, 4, 2, 1, bias=False))
            main.add_module('pyramid.{0}.batchnorm'.format(out_feat),
                            nn.BatchNorm2d(out_feat))
            main.add_module('pyramid.{0}.relu'.format(out_feat),
                            nn.LeakyReLU(0.2, inplace=True))
            cndf = cndf * 2
            csize = csize / 2

        # state size. K x 4 x 4
        main.add_module('final.{0}-{1}.conv'.format(cndf, 1),
                        nn.Conv2d(cndf, 1, 4, 1, 0, bias=False))
        self.main = main


    def forward(self, input):
        if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else: 
            output = self.main(input)
            
        output = output.mean(0)
        return output.view(1)

In [200]:
class DCGAN_G(nn.Module):
    def __init__(self, isize, nz, nc, ngf, ngpu, n_extra_layers=0):
        super(DCGAN_G, self).__init__()
        self.ngpu = ngpu
        assert isize % 16 == 0, "isize has to be a multiple of 16"

        cngf, tisize = ngf//2, 4
        while tisize != isize:
            cngf = cngf * 2
            tisize = tisize * 2

        main = nn.Sequential()
        # input is Z, going into a convolution
        main.add_module('initial.{0}-{1}.convt'.format(nz, cngf),
                        nn.ConvTranspose2d(nz, cngf, 4, 1, 0, bias=False))
        main.add_module('initial.{0}.batchnorm'.format(cngf),
                        nn.BatchNorm2d(cngf))
        main.add_module('initial.{0}.relu'.format(cngf),
                        nn.ReLU(True))

        csize, cndf = 4, cngf
        while csize < isize//2:
            main.add_module('pyramid.{0}-{1}.convt'.format(cngf, cngf//2),
                            nn.ConvTranspose2d(cngf, cngf//2, 4, 2, 1, bias=False))
            main.add_module('pyramid.{0}.batchnorm'.format(cngf//2),
                            nn.BatchNorm2d(cngf//2))
            main.add_module('pyramid.{0}.relu'.format(cngf//2),
                            nn.ReLU(True))
            cngf = cngf // 2
            csize = csize * 2

        # Extra layers
        for t in range(n_extra_layers):
            main.add_module('extra-layers-{0}.{1}.conv'.format(t, cngf),
                            nn.Conv2d(cngf, cngf, 3, 1, 1, bias=False))
            main.add_module('extra-layers-{0}.{1}.batchnorm'.format(t, cngf),
                            nn.BatchNorm2d(cngf))
            main.add_module('extra-layers-{0}.{1}.relu'.format(t, cngf),
                            nn.ReLU(True))

        main.add_module('final.{0}-{1}.convt'.format(cngf, nc),
                        nn.ConvTranspose2d(cngf, nc, 4, 2, 1, bias=False))
        main.add_module('final.{0}.tanh'.format(nc),
                        nn.Tanh())
        self.main = main

    def forward(self, input):
        if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else: 
            output = self.main(input)
        return output 

In [201]:
ngpu = 1 # number of GPUs to use
nz = 100 # size of the latent z vector
ngf = 512
ndf = 512
nc = 1 # input image channels
n_extra_layers = 0 # Number of extra layers on gen and disc

imageSize = 28
batchSize = 100
n_workers = 2


adam = False
lrD = 0.00005
lrG = 0.00005

beta1 = 0.5 # beta1 for adam. default=0.5
niter = 500 # number of epochs to train for
Diters = 5 #number of D iters per each G iter



clamp_lower = -0.01
clamp_upper = 0.01

experiment = './experiment' # Where to store samples and models

epsilon = 1 # panalty weight
L = 10 # sinkhorn iteration num

netG_path = ''
netD_path = ''

In [202]:
# get data
dataset = dset.MNIST(root='./data', download=True,
                           transform=transforms.Compose([
                               transforms.Scale(imageSize),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))

dataloader = torch.utils.data.DataLoader(dataset, batch_size=batchSize,
                                         shuffle=True, num_workers=n_workers)

In [203]:
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [204]:
netG = MLP_G(imageSize, nz, nc, ngf, ngpu)
netD = MLP_D(imageSize, nz, nc, ndf, ngpu)

netG.apply(weights_init)
if netG_path != '': # load checkpoint if needed
    netG.load_state_dict(torch.load(netG_path))
print(netG)

netD.apply(weights_init)
if netD_path != '':
    netD.load_state_dict(torch.load(netD_path))
print(netD)

MLP_G (
  (main): Sequential (
    (0): Linear (100 -> 512)
    (1): ReLU (inplace)
    (2): Linear (512 -> 512)
    (3): ReLU (inplace)
    (4): Linear (512 -> 512)
    (5): ReLU (inplace)
    (6): Linear (512 -> 784)
  )
)
MLP_D (
  (main): Sequential (
    (0): Linear (784 -> 512)
    (1): ReLU (inplace)
    (2): Linear (512 -> 512)
    (3): ReLU (inplace)
    (4): Linear (512 -> 512)
    (5): ReLU (inplace)
    (6): Linear (512 -> 1)
  )
)


In [205]:
input = torch.FloatTensor(batchSize, 3, imageSize, imageSize)
noise = torch.FloatTensor(batchSize, nz, 1, 1)
fixed_noise = torch.FloatTensor(batchSize, nz, 1, 1).normal_(0, 1)
one = torch.FloatTensor([1])
mone = one * -1

In [206]:
if adam:
    optimizerD = optim.Adam(netD.parameters(), lr=lrD, betas=(beta1, 0.999))
    optimizerG = optim.Adam(netG.parameters(), lr=lrG, betas=(beta1, 0.999))
else:
    optimizerD = optim.RMSprop(netD.parameters(), lr = lrD)
    optimizerG = optim.RMSprop(netG.parameters(), lr = lrG)

In [ ]:
from IPython.core.debugger import Tracer
gen_iterations = 0
for epoch in range(niter):
    #Tracer()()
    data_iter = iter(dataloader)
    i = 0
    while i < len(dataloader):
        ############################
        # (1) Update D network
        ###########################
        for p in netD.parameters(): # reset requires_grad
            p.requires_grad = True # they are set to False below in netG update

        # train the discriminator Diters times
        if gen_iterations < 25 or gen_iterations % 500 == 0:
            Diters = 100
        #else:
        #    Diters = opt.Diters
        j = 0
        while j < Diters and i < len(dataloader):
            j += 1

            # clamp parameters to a cube
            for p in netD.parameters():
                p.data.clamp_(clamp_lower, clamp_upper)

            data = data_iter.next()
            i += 1

            # train with real
            real_cpu, _ = data
            netD.zero_grad()
            batch_size = real_cpu.size(0)

            #if opt.cuda:
            #    real_cpu = real_cpu.cuda()
            input.resize_as_(real_cpu).copy_(real_cpu)
            inputv = Variable(input)

            errD_real = netD(inputv)
            errD_real.backward(one)

            # train with fake
            noise.resize_(batchSize, nz, 1, 1).normal_(0, 1)
            noisev = Variable(noise, volatile = True) # totally freeze netG
            fake = Variable(netG(noisev).data)
            inputv = fake
            errD_fake = netD(inputv)
            errD_fake.backward(mone)
            errD = errD_real - errD_fake
            optimizerD.step()

        ############################
        # (2) Update G network
        ###########################
        for p in netD.parameters():
            p.requires_grad = False # to avoid computation
        netG.zero_grad()
        # in case our last batch was the tail batch of the dataloader,
        # make sure we feed a full batch of noise
        noise.resize_(batchSize, nz, 1, 1).normal_(0, 1)
        noisev = Variable(noise)
        fake = netG(noisev)
        errG = netD(fake)
        errG.backward(one)
        optimizerG.step()
        gen_iterations += 1

        print('[%d/%d][%d/%d][%d] Loss_D: %f Loss_G: %f Loss_D_real: %f Loss_D_fake %f'
            % (epoch, niter, i, len(dataloader), gen_iterations,
            errD.data[0], errG.data[0], errD_real.data[0], errD_fake.data[0]))
        if gen_iterations % 100 == 0:
            real_cpu = real_cpu.mul(0.5).add(0.5)
            vutils.save_image(real_cpu, '{0}/real_samples.png'.format(experiment))
            fake = netG(Variable(fixed_noise, volatile=True))
            fake.data = fake.data.mul(0.5).add(0.5)
            vutils.save_image(fake.data, '{0}/fake_samples_{1}.png'.format(experiment, gen_iterations))

    # do checkpointing
    torch.save(netG.state_dict(), '{0}/WGANnetG_epoch_{1}.pth'.format(experiment, epoch))
    torch.save(netD.state_dict(), '{0}/WGANnetD_epoch_{1}.pth'.format(experiment, epoch))

[0/500][100/600][1] Loss_D: -82.874374 Loss_G: 0.401282 Loss_D_real: -82.494942 Loss_D_fake 0.379434
[0/500][200/600][2] Loss_D: -134.639618 Loss_G: 0.457681 Loss_D_real: -134.198166 Loss_D_fake 0.441453
[0/500][300/600][3] Loss_D: -136.300064 Loss_G: 0.372870 Loss_D_real: -135.934799 Loss_D_fake 0.365270
[0/500][400/600][4] Loss_D: -134.044525 Loss_G: 0.309708 Loss_D_real: -133.740906 Loss_D_fake 0.303615
[0/500][500/600][5] Loss_D: -137.306717 Loss_G: 0.269333 Loss_D_real: -137.043732 Loss_D_fake 0.262984
[0/500][600/600][6] Loss_D: -136.274200 Loss_G: 0.198808 Loss_D_real: -136.069382 Loss_D_fake 0.204811
[1/500][100/600][7] Loss_D: -132.403412 Loss_G: -1.935215 Loss_D_real: -134.298874 Loss_D_fake -1.895460
[1/500][200/600][8] Loss_D: -132.586182 Loss_G: -5.227728 Loss_D_real: -137.727982 Loss_D_fake -5.141803
[1/500][300/600][9] Loss_D: -128.526230 Loss_G: -9.796119 Loss_D_real: -138.208130 Loss_D_fake -9.681898
[1/500][400/600][10] Loss_D: -118.496674 Loss_G: -16.441322 Loss_D_re

[13/500][400/600][82] Loss_D: -15.976317 Loss_G: -3.708876 Loss_D_real: -19.718325 Loss_D_fake -3.742007
[13/500][500/600][83] Loss_D: -15.072958 Loss_G: -4.125761 Loss_D_real: -18.815689 Loss_D_fake -3.742731
[13/500][600/600][84] Loss_D: -17.572254 Loss_G: -4.168663 Loss_D_real: -21.576838 Loss_D_fake -4.004583
[14/500][100/600][85] Loss_D: -15.178810 Loss_G: -3.602073 Loss_D_real: -18.767057 Loss_D_fake -3.588247
[14/500][200/600][86] Loss_D: -15.745811 Loss_G: -3.739495 Loss_D_real: -19.756601 Loss_D_fake -4.010790
[14/500][300/600][87] Loss_D: -15.744574 Loss_G: -3.811072 Loss_D_real: -19.634260 Loss_D_fake -3.889687
[14/500][400/600][88] Loss_D: -15.428824 Loss_G: -3.877462 Loss_D_real: -18.890284 Loss_D_fake -3.461459
[14/500][500/600][89] Loss_D: -16.933134 Loss_G: -3.091544 Loss_D_real: -20.289600 Loss_D_fake -3.356466
[14/500][600/600][90] Loss_D: -15.204451 Loss_G: -3.292985 Loss_D_real: -18.419872 Loss_D_fake -3.215422
[15/500][100/600][91] Loss_D: -16.314600 Loss_G: -3.005

[26/500][400/600][160] Loss_D: -13.904155 Loss_G: 1.117943 Loss_D_real: -12.941466 Loss_D_fake 0.962688
[26/500][500/600][161] Loss_D: -13.398342 Loss_G: 0.902044 Loss_D_real: -12.320239 Loss_D_fake 1.078103
[26/500][600/600][162] Loss_D: -11.265482 Loss_G: 1.090268 Loss_D_real: -10.180790 Loss_D_fake 1.084692
[27/500][100/600][163] Loss_D: -12.749969 Loss_G: 0.799912 Loss_D_real: -11.958366 Loss_D_fake 0.791602
[27/500][200/600][164] Loss_D: -13.597390 Loss_G: -0.009459 Loss_D_real: -13.565852 Loss_D_fake 0.031538
[27/500][300/600][165] Loss_D: -15.646103 Loss_G: 1.077305 Loss_D_real: -15.483196 Loss_D_fake 0.162906
[27/500][400/600][166] Loss_D: -13.719656 Loss_G: -0.290453 Loss_D_real: -13.520429 Loss_D_fake 0.199228
[27/500][500/600][167] Loss_D: -14.846827 Loss_G: 2.018766 Loss_D_real: -13.526970 Loss_D_fake 1.319856
[27/500][600/600][168] Loss_D: -13.906822 Loss_G: 1.401554 Loss_D_real: -12.729840 Loss_D_fake 1.176981
[28/500][100/600][169] Loss_D: -12.284311 Loss_G: 1.018830 Los

[39/500][400/600][238] Loss_D: -10.820286 Loss_G: -1.053818 Loss_D_real: -12.183001 Loss_D_fake -1.362715
[39/500][500/600][239] Loss_D: -11.166741 Loss_G: -0.985909 Loss_D_real: -11.830677 Loss_D_fake -0.663936
[39/500][600/600][240] Loss_D: -11.608454 Loss_G: 0.641473 Loss_D_real: -10.812588 Loss_D_fake 0.795866
[40/500][100/600][241] Loss_D: -11.255083 Loss_G: 0.640813 Loss_D_real: -10.408608 Loss_D_fake 0.846474
[40/500][200/600][242] Loss_D: -11.485519 Loss_G: -1.061342 Loss_D_real: -11.854192 Loss_D_fake -0.368672
[40/500][300/600][243] Loss_D: -11.865269 Loss_G: -0.362820 Loss_D_real: -11.352063 Loss_D_fake 0.513205
[40/500][400/600][244] Loss_D: -11.082159 Loss_G: 1.495555 Loss_D_real: -9.086387 Loss_D_fake 1.995773
[40/500][500/600][245] Loss_D: -12.950786 Loss_G: 3.020081 Loss_D_real: -10.809839 Loss_D_fake 2.140947
[40/500][600/600][246] Loss_D: -11.426716 Loss_G: 1.653224 Loss_D_real: -9.145351 Loss_D_fake 2.281364
[41/500][100/600][247] Loss_D: -11.643253 Loss_G: 3.085781 

[52/500][500/600][317] Loss_D: -12.395195 Loss_G: 0.550566 Loss_D_real: -11.122878 Loss_D_fake 1.272317
[52/500][600/600][318] Loss_D: -12.290512 Loss_G: 1.422979 Loss_D_real: -10.081714 Loss_D_fake 2.208798
[53/500][100/600][319] Loss_D: -13.383944 Loss_G: 3.808830 Loss_D_real: -10.785015 Loss_D_fake 2.598928
[53/500][200/600][320] Loss_D: -12.745975 Loss_G: 1.971069 Loss_D_real: -11.157339 Loss_D_fake 1.588637
[53/500][300/600][321] Loss_D: -12.032018 Loss_G: -0.194311 Loss_D_real: -11.337743 Loss_D_fake 0.694275
[53/500][400/600][322] Loss_D: -11.399744 Loss_G: -0.136208 Loss_D_real: -11.936398 Loss_D_fake -0.536654
[53/500][500/600][323] Loss_D: -11.406766 Loss_G: -2.387017 Loss_D_real: -12.125616 Loss_D_fake -0.718850
[53/500][600/600][324] Loss_D: -12.953060 Loss_G: -0.011139 Loss_D_real: -13.203300 Loss_D_fake -0.250240
[54/500][100/600][325] Loss_D: -11.929152 Loss_G: -1.070486 Loss_D_real: -13.072313 Loss_D_fake -1.143162
[54/500][200/600][326] Loss_D: -11.588348 Loss_G: -2.18

...

[157/500][100/600][943] Loss_D: -8.907434 Loss_G: 1.732522 Loss_D_real: -7.729695 Loss_D_fake 1.177739
[157/500][200/600][944] Loss_D: -8.901672 Loss_G: 0.415489 Loss_D_real: -8.683837 Loss_D_fake 0.217835
[157/500][300/600][945] Loss_D: -10.071474 Loss_G: -0.937472 Loss_D_real: -10.541346 Loss_D_fake -0.469872
[157/500][400/600][946] Loss_D: -9.789824 Loss_G: -0.989962 Loss_D_real: -10.849593 Loss_D_fake -1.059769
[157/500][500/600][947] Loss_D: -8.852380 Loss_G: -0.505669 Loss_D_real: -9.811206 Loss_D_fake -0.958826
[157/500][600/600][948] Loss_D: -10.726007 Loss_G: 0.878643 Loss_D_real: -11.491255 Loss_D_fake -0.765248
[158/500][100/600][949] Loss_D: -9.698827 Loss_G: -0.023662 Loss_D_real: -10.283413 Loss_D_fake -0.584586
[158/500][200/600][950] Loss_D: -8.778839 Loss_G: -4.111264 Loss_D_real: -10.775208 Loss_D_fake -1.996369
[158/500][300/600][951] Loss_D: -9.555592 Loss_G: 1.081293 Loss_D_real: -9.395956 Loss_D_fake 0.159636
[158/500][400/600][952] Loss_D: -7.986349 Loss_G: 0.221

[170/500][200/600][1022] Loss_D: -9.359047 Loss_G: -1.341268 Loss_D_real: -10.341316 Loss_D_fake -0.982269
[170/500][300/600][1023] Loss_D: -10.058031 Loss_G: -0.474053 Loss_D_real: -10.037220 Loss_D_fake 0.020811
[170/500][400/600][1024] Loss_D: -9.185861 Loss_G: -0.185306 Loss_D_real: -9.355678 Loss_D_fake -0.169817
[170/500][500/600][1025] Loss_D: -9.057310 Loss_G: -0.567746 Loss_D_real: -10.059293 Loss_D_fake -1.001982
[170/500][600/600][1026] Loss_D: -9.045682 Loss_G: -1.322665 Loss_D_real: -10.203561 Loss_D_fake -1.157879
[171/500][100/600][1027] Loss_D: -8.973425 Loss_G: -2.788847 Loss_D_real: -10.101490 Loss_D_fake -1.128066
[171/500][200/600][1028] Loss_D: -9.672999 Loss_G: 1.569776 Loss_D_real: -8.533741 Loss_D_fake 1.139258
[171/500][300/600][1029] Loss_D: -9.081300 Loss_G: 0.781745 Loss_D_real: -8.557337 Loss_D_fake 0.523963
[171/500][400/600][1030] Loss_D: -8.996534 Loss_G: 0.344758 Loss_D_real: -9.069684 Loss_D_fake -0.073150
[171/500][500/600][1031] Loss_D: -9.198098 Los

[183/500][200/600][1100] Loss_D: -8.334165 Loss_G: -0.542111 Loss_D_real: -9.659250 Loss_D_fake -1.325086
[183/500][300/600][1101] Loss_D: -8.566017 Loss_G: -0.695675 Loss_D_real: -9.414930 Loss_D_fake -0.848913
[183/500][400/600][1102] Loss_D: -7.754129 Loss_G: -0.168630 Loss_D_real: -7.925100 Loss_D_fake -0.170971
[183/500][500/600][1103] Loss_D: -8.536405 Loss_G: 0.612463 Loss_D_real: -9.564494 Loss_D_fake -1.028090
[183/500][600/600][1104] Loss_D: -7.362186 Loss_G: -2.065177 Loss_D_real: -9.561790 Loss_D_fake -2.199603
[184/500][100/600][1105] Loss_D: -7.503172 Loss_G: -2.594247 Loss_D_real: -8.935297 Loss_D_fake -1.432125
[184/500][200/600][1106] Loss_D: -9.060043 Loss_G: 0.114331 Loss_D_real: -8.474626 Loss_D_fake 0.585418
[184/500][300/600][1107] Loss_D: -8.192348 Loss_G: 1.745807 Loss_D_real: -6.955822 Loss_D_fake 1.236526
[184/500][400/600][1108] Loss_D: -8.354540 Loss_G: 0.237301 Loss_D_real: -7.566301 Loss_D_fake 0.788239
[184/500][500/600][1109] Loss_D: -8.000475 Loss_G: 0.

[196/500][200/600][1178] Loss_D: -8.601271 Loss_G: 2.187176 Loss_D_real: -7.106926 Loss_D_fake 1.494344
[196/500][300/600][1179] Loss_D: -8.559580 Loss_G: 2.754629 Loss_D_real: -6.291803 Loss_D_fake 2.267776
[196/500][400/600][1180] Loss_D: -7.777126 Loss_G: 1.712613 Loss_D_real: -6.627399 Loss_D_fake 1.149726
[196/500][500/600][1181] Loss_D: -8.288447 Loss_G: -0.532914 Loss_D_real: -7.460742 Loss_D_fake 0.827706
[196/500][600/600][1182] Loss_D: -8.193974 Loss_G: 1.076391 Loss_D_real: -9.557452 Loss_D_fake -1.363477
[197/500][100/600][1183] Loss_D: -8.356319 Loss_G: -2.770743 Loss_D_real: -9.318389 Loss_D_fake -0.962069
[197/500][200/600][1184] Loss_D: -8.518054 Loss_G: 1.145327 Loss_D_real: -8.299462 Loss_D_fake 0.218592
[197/500][300/600][1185] Loss_D: -8.166758 Loss_G: -0.747379 Loss_D_real: -7.968142 Loss_D_fake 0.198616
[197/500][400/600][1186] Loss_D: -8.417742 Loss_G: 1.069754 Loss_D_real: -7.143020 Loss_D_fake 1.274722
[197/500][500/600][1187] Loss_D: -8.740047 Loss_G: 1.789481

[209/500][300/600][1257] Loss_D: -9.763142 Loss_G: -2.730641 Loss_D_real: -10.924882 Loss_D_fake -1.161740
[209/500][400/600][1258] Loss_D: -7.911051 Loss_G: -1.155771 Loss_D_real: -7.697009 Loss_D_fake 0.214043
[209/500][500/600][1259] Loss_D: -8.213019 Loss_G: 1.324682 Loss_D_real: -7.562060 Loss_D_fake 0.650959
[209/500][600/600][1260] Loss_D: -8.040619 Loss_G: -0.349805 Loss_D_real: -7.794613 Loss_D_fake 0.246006
[210/500][100/600][1261] Loss_D: -8.349628 Loss_G: -0.281458 Loss_D_real: -8.105285 Loss_D_fake 0.244344
[210/500][200/600][1262] Loss_D: -8.765704 Loss_G: 1.330974 Loss_D_real: -7.877647 Loss_D_fake 0.888057
[210/500][300/600][1263] Loss_D: -8.199166 Loss_G: 0.408548 Loss_D_real: -8.027755 Loss_D_fake 0.171411
[210/500][400/600][1264] Loss_D: -8.579100 Loss_G: 0.802683 Loss_D_real: -8.608553 Loss_D_fake -0.029454
[210/500][500/600][1265] Loss_D: -8.700569 Loss_G: -0.967818 Loss_D_real: -8.536190 Loss_D_fake 0.164379
[210/500][600/600][1266] Loss_D: -8.023254 Loss_G: 0.223

In [ ]:
noise.resize_(16, nz, 1, 1).normal_(0, 1)
noisev = Variable(noise)
fake = netG(noisev)
fake = fake.mul(0.5).add(0.5)
fake = fake - fake.min()
import matplotlib.pyplot as plt
from torchvision import utils
def imshow(inp, save=False, title=None):
    """Imshow for Tensor."""
    fig = plt.figure(figsize=(5, 5))
    inp = inp.numpy().transpose((1, 2, 0))
    plt.imshow(inp,cmap='gray')
    plt.show()
dd = utils.make_grid(fake.data)
imshow(dd)